#### 문제 2-4 : 여행 계획 분석기

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_w


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# 출력 구조 정의 (여행지, 기간, 예산, 추천도, 주요 활동 리스트)
response_schemas = [
    ResponseSchema(name="destination", description="여행한 장소 (예: 부산, 제주도)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일)"),
    ResponseSchema(name="budget", description="여행 예산 (예: 30만원)"),
    ResponseSchema(name="rating", description="여행 만족도 (1~5점 중 하나, 숫자만 출력)"),
    ResponseSchema(name="activities", description="여행 중 주요 활동 리스트. 반드시 문자열 배열로 출력 (예: ['해운대 바다구경', '자갈치시장 회 먹기'])")
]

In [14]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("여행 계획 분석기 출력 형식 지시사항:")
print(format_instructions)

여행 계획 분석기 출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행한 장소 (예: 부산, 제주도)
	"duration": string  // 여행 기간 (예: 2박 3일)
	"budget": string  // 여행 예산 (예: 30만원)
	"rating": string  // 여행 만족도 (1~5점 중 하나, 숫자만 출력)
	"activities": string  // 여행 중 주요 활동 리스트. 반드시 문자열 배열로 출력 (예: ['해운대 바다구경', '자갈치시장 회 먹기'])
}
```


In [15]:
# 프롬프트 템플릿
template = """
당신은 여행 계획 분석 전문가입니다.
다음 여행 후기 또는 계획 텍스트에서 핵심 정보를 추출하여 JSON 형식으로 구조화하세요.

1. rating은 반드시 숫자만 출력하세요 (예: "4")
2. activities는 반드시 문자열 리스트로 출력하세요 (예: ["해운대 바다구경", "자갈치시장 회 먹기"])

여행 텍스트: {travel_text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [16]:
# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [17]:
# 테스트 여행 후기 데이터
travel_text = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 
감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

In [18]:
# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"travel_text": travel_text})

# 결과 출력 (Pretty Print)
print("===== 여행 분석 결과 =====")
pprint(output)

===== 여행 분석 결과 =====
{'activities': ['해운대 바다구경', '자갈치시장 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': 4}
